In [11]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
 
MODEL_NAME = "TheBloke/Llama-2-13b-Chat-GPTQ"
 
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
 
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto"
)
 
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15
 
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
)
 
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

/root/miniconda3/envs/langchain_tut/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
tokenizer_config.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 727/727 [00:00<00:00, 1.15MB/s]
tokenizer.model: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500k/500k [00:00<00:00, 42.1MB/s]
tokenizer.json: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1.84M/1.84M [00:00<00:00, 30.5MB/s]
config.json: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████

In [12]:
result = llm(
    "Explain the difference between ChatGPT and open source LLMs in a couple of lines."
)
print(result)



Answer: Sure! Here's the difference between ChatGPT and open-source large language models (LLMs) in two lines:

ChatGPT is a proprietary, closed-source AI model developed by Meta AI that offers a more user-friendly interface and seamless integration with other Meta products, while open-source LLMs like BERT and RoBERTa are freely available for anyone to use and modify, but may require more technical expertise to integrate into applications.


In [13]:
from langchain import PromptTemplate
 
template = """
<s>[INST] <<SYS>>
Act as a Machine Learning engineer who is teaching high school students.
<</SYS>>
 
{text} [/INST]
"""
 
prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [14]:
text = "Explain what are Deep Neural Networks in 2-3 sentences"
print(prompt.format(text=text))


<s>[INST] <<SYS>>
Act as a Machine Learning engineer who is teaching high school students.
<</SYS>>
 
Explain what are Deep Neural Networks in 2-3 sentences [/INST]



In [15]:
from langchain.chains import LLMChain
 
chain = LLMChain(llm=llm, prompt=prompt)
result = chain.run(text)
print(result)


Hey there, young minds! So, you wanna know about Deep Neural Networks? Well, imagine you have a super powerful computer that can learn and make decisions all on its own, kinda like how your brain works! Deep Neural Networks are like a bunch of these computers working together to solve really tough problems, like recognizing pictures or understanding speech. They're like the ultimate team players, and they're changing the game in fields like self-driving cars, medical diagnosis, and more!


In [16]:
template = "<s>[INST] Use the summary {summary} and give 3 examples of practical applications with 1 sentence explaining each [/INST]"
 
examples_prompt = PromptTemplate(
    input_variables=["summary"],
    template=template,
)
examples_chain = LLMChain(llm=llm, prompt=examples_prompt)

In [17]:
from langchain.chains import SimpleSequentialChain
 
multi_chain = SimpleSequentialChain(chains=[chain, examples_chain], verbose=True)
result = multi_chain.run(text)
print(result.strip())



> Entering new SimpleSequentialChain chain...

Hey there, young minds! So, you wanna know about Deep Neural Networks? Well, imagine you have a super powerful computer that can learn and make decisions all on its own, kinda like how your brain works! Deep Neural Networks are like a bunch of these computers working together to solve really tough problems, like recognizing pictures or understanding speech. They're like the ultimate team players, and they're changing the game in fields like self-driving cars, medical diagnosis, and more!
  Sure thing! Here are three examples of practical applications of Deep Neural Networks:

1. Self-Driving Cars: Deep Neural Networks can be used to train autonomous vehicles to recognize objects on the road, such as pedestrians, other cars, and traffic lights, allowing them to make safe and efficient decisions.
2. Medical Diagnosis: Deep Neural Networks can be trained on large datasets of medical images and patient data to help doctors diagnose diseases 

In [21]:
from langchain.document_loaders import UnstructuredMarkdownLoader
 
loader = UnstructuredMarkdownLoader("bitcoin.md")
docs = loader.load()
len(docs)

1

In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

38

In [23]:
from langchain.embeddings import HuggingFaceEmbeddings
 
embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)
 
query_result = embeddings.embed_query(texts[0].page_content)
print(len(query_result))

.gitattributes: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1.52k/1.52k [00:00<00:00, 2.20MB/s]
1_Pooling/config.json: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 191/191 [00:00<00:00, 325kB/s]
README.md: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 67.9k/67.9k [00:00<00:00, 781kB/s]
config.json: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 619/619 [00:00<00:00, 986kB/s]
model.safetensors: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 670M/670M [00:16<00:00, 39.5MB/s]


1024


In [24]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(texts, embeddings, persist_directory="db")
results = db.similarity_search("proof-of-work majority decision making", k=2)
print(results[0].page_content)

we implement the proof-of-work by\nincrementing a nonce in the block until a value is found that gives the\nblock's hash the required zero bits. Once the CPU effort has been\nexpended to make it satisfy the proof-of-work, the block cannot be\nchanged without redoing the work. As later blocks are chained after it,\nthe work to change the block would include redoing all the blocks after\nit\n\nThe proof-of-work also solves the problem of determining representation\nin majority decision making. If the majority were based on\none-IP-address-one-vote, it could be subverted by anyone able to\nallocate many IPs. Proof-of-work is essentially one-CPU-one-vote. The\nmajority decision is represented by the longest chain, which has the\ngreatest proof-of-work effort invested in it. If a majority of CPU power\nis controlled by honest nodes, the honest chain will grow the fastest\nand outpace any competing chains. To modify a past block, an attacker\nwould have to redo the proof-of-work of the block

In [25]:
from langchain.chains import RetrievalQA
 
template = """
<s>[INST] <<SYS>>
Act as a cryptocurrency expert. Use the following information to answer the question at the end.
<</SYS>>
 
{context}
 
{question} [/INST]
"""
 
prompt = PromptTemplate(template=template, input_variables=["context", "question"])
 
 
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)
 
result = qa_chain(
    "How does proof-of-work solves the majority decision making problem? Explain like I am five."
)
print(result["result"].strip())

Okay, little buddy! So you know how there are lots of different people in the world, right? And sometimes, these people might not agree on things, like what game to play or what candy to eat. Well, imagine that instead of people, we have computers that talk to each other and do things together. These computers are like really smart and can do lots of things fast, like calculate numbers really quickly!

Now, when these computers want to decide something, like which game to play or which candy to eat, they need to figure out who gets to decide. And that's where proof-of-work comes in! It's like a special trick that helps the computers decide who gets to say what.

Here's how it works: when a computer wants to decide something, it has to show everyone else that it did some work first. Like, it has to solve a really hard math problem that takes a long time to solve. And once it solves that problem, it gets to say what it wants to say! But here's the cool part: if another computer comes alo

In [26]:
from textwrap import fill
 
result = qa_chain(
    "Summarize the privacy compared to the traditional banking model in 2-3 sentences."
)
print(fill(result["result"].strip(), width=80))

As an expert in cryptocurrency, I can explain that the privacy offered by
Bitcoin's decentralized ledger technology is significantly better than the
traditional banking model. In the traditional banking model, financial
transactions are recorded on a centralized ledger controlled by a single entity,
which can potentially compromise the privacy of users. In contrast, Bitcoin's
decentralized ledger technology allows for the recording of transactions without
revealing the identities of the parties involved, providing greater privacy and
anonymity for users. Additionally, the use of public keys instead of real names
further enhances the privacy features of the network.
